In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext(appName='3_Clustering')

In [4]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors, Matrices
import numpy as np

In [5]:
from pyspark.mllib.util import MLUtils
path= '/home/praneeth/Downloads/Case3_TvNews/BBC.txt'
points = MLUtils.loadLibSVMFile(sc,path)

In [6]:

cnn = "/home/praneeth/Downloads/Case3_TvNews/CNN.txt"
cnn1 = MLUtils.loadLibSVMFile(sc, cnn)
x = points.union(cnn1)


cnnibn = "/home/praneeth/Downloads/Case3_TvNews/CNNIBN.txt"
cnnibn1 = MLUtils.loadLibSVMFile(sc, cnnibn)


y = x.union(cnnibn1)


ndtv = "/home/praneeth/Downloads/Case3_TvNews/NDTV.txt"
ndtv1 = MLUtils.loadLibSVMFile(sc, ndtv)


z = y.union(ndtv1)


tn = "/home/praneeth/Downloads/Case3_TvNews/TIMESNOW.txt"
tn1 = MLUtils.loadLibSVMFile(sc, tn)

data = z.union(tn1)


data.count()

129685

In [31]:
data.take(2)

[LabeledPoint(1.0, (4125,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,89,90,91,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,109,110,111,112,114,115,116,117,118,122,218,275,295,447,490,571,572,600,622,725,761,815,870,923,958,1001,1015,1047,4123,4124],[123.0,1.31644,1.516003,5.605905,5.34676,0.013233,0.010729,0.091743,0.050768,3808.067871,702.992493,7533.133301,1390.499268,971.098511,1894.978027,114.965019,45.018257,0.635224,0.095226,0.063398,0.06121,0.038319,0.018285,0.011113,0.007736,0.004864,0.00422,0.003273,0.002699,0.002553,0.002323,0.002108,0.002036,0.001792,0.001553,0.00125,0.001317,0.001084,0.000818,0.000624,0.000586,0.000529,0.000426,0.000359,0.000446,0.000268,0.000221,0.000154,0.000217,0.000193,0.000163,0.000165,0.00021,0.000114,0.00013,5.5e-05,1.3e-05,0.733037,0.13

In [7]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.feature import StandardScaler

In [8]:
features = data.map(lambda x: x.features)

In [9]:
scaler2 = StandardScaler(withMean=True, withStd=True).fit(features)

In [10]:
data2 = scaler2.transform(features.map(lambda x: Vectors.dense(x.toArray()))).cache()

In [44]:
data2.first()

DenseVector([0.0627, -0.5828, -0.0619, -0.6961, -0.4261, -0.3654, 0.2949, -0.308, -0.2791, 0.4879, -0.8566, 0.5413, -0.8096, -0.2344, 0.6458, 0.0067, -0.2994, 0.6306, -0.0597, 0.3679, 1.0282, 0.5459, -0.1918, -0.4535, -0.5799, -0.6727, -0.6744, -0.6957, -0.7213, -0.678, -0.6484, -0.6033, -0.5518, -0.5219, -0.5246, -0.5307, -0.4606, -0.4147, -0.4893, -0.4937, -0.4096, -0.3992, -0.4408, -0.4054, -0.3527, -0.4207, -0.4145, -0.4353, -0.3675, -0.3501, -0.3579, -0.3324, -0.2159, -0.3154, -0.1971, -0.3357, -0.3936, 0.9365, -0.116, -0.3485, -0.5389, -0.6468, -0.7321, -0.7856, -0.7942, -0.7983, -0.7927, -0.7804, -0.7726, -0.749, -0.7268, -0.6925, -0.6747, -0.6349, -0.5817, -0.548, -0.4851, -0.4596, -0.3936, -0.3516, -0.327, -0.2495, -0.1233, -0.1621, -0.0764, 1.1891, -0.2161, 0.0, 0.0, -0.5076, 0.5466, 0.2273, -0.0869, 0.4605, -0.009, -0.5136, -0.5839, -0.514, -0.7723, -0.7699, -0.698, 0.1779, 0.4709, -0.5843, -0.4419, 1.1958, -0.1248, -0.0357, 1.1534, 0.4414, -0.4925, -0.3076, -0.4575, -0.8784

In [13]:
from pyspark.mllib.clustering import KMeans

#Build the model (cluster the data)
clusters = KMeans.train(data2, 4, maxIterations=10,
        runs=10, initializationMode="random")

In [15]:
# Evaluate clustering by computing Within Set Sum of Squared Errors
from math import sqrt
from numpy import array
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))
WSSSE = data2.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

Within Set Sum of Squared Error = 1515285.453


In [16]:
for k in [6,8,10]:
    clusters = KMeans.train(data2, k=k, maxIterations=10,
        runs=10, initializationMode="random")
    WSSSE = data2.map(lambda point: error(point)).reduce(lambda x, y: x + y)
    print("Within Set Sum of Squared Error = " + str(WSSSE))


Within Set Sum of Squared Error = 1470567.11634
Within Set Sum of Squared Error = 1438714.29691
Within Set Sum of Squared Error = 1425614.81305


In [17]:
from pyspark.mllib.clustering import GaussianMixture
from numpy import array

In [20]:
#parsedData = data2.map(lambda line: array([float(x) for x in line.split(' ')]))
#Vectors.dense(x.toArray()

In [33]:
parsedData = data.map(lambda line:line.features)

In [34]:
parsedData.take(2)

[SparseVector(4125, {0: 123.0, 1: 1.3164, 2: 1.516, 3: 5.6059, 4: 5.3468, 5: 0.0132, 6: 0.0107, 7: 0.0917, 8: 0.0508, 9: 3808.0679, 10: 702.9925, 11: 7533.1333, 12: 1390.4993, 13: 971.0985, 14: 1894.978, 15: 114.965, 16: 45.0183, 17: 0.6352, 18: 0.0952, 19: 0.0634, 20: 0.0612, 21: 0.0383, 22: 0.0183, 23: 0.0111, 24: 0.0077, 25: 0.0049, 26: 0.0042, 27: 0.0033, 28: 0.0027, 29: 0.0026, 30: 0.0023, 31: 0.0021, 32: 0.002, 33: 0.0018, 34: 0.0016, 35: 0.0013, 36: 0.0013, 37: 0.0011, 38: 0.0008, 39: 0.0006, 40: 0.0006, 41: 0.0005, 42: 0.0004, 43: 0.0004, 44: 0.0004, 45: 0.0003, 46: 0.0002, 47: 0.0002, 48: 0.0002, 49: 0.0002, 50: 0.0002, 51: 0.0002, 52: 0.0002, 53: 0.0001, 54: 0.0001, 55: 0.0001, 56: 0.0, 57: 0.733, 58: 0.1331, 59: 0.0413, 60: 0.0197, 61: 0.011, 62: 0.0069, 63: 0.0045, 64: 0.0031, 65: 0.0023, 66: 0.0018, 67: 0.0014, 68: 0.0011, 69: 0.0009, 70: 0.0008, 71: 0.0007, 72: 0.0006, 73: 0.0005, 74: 0.0005, 75: 0.0005, 76: 0.0005, 77: 0.0004, 78: 0.0004, 79: 0.0003, 80: 0.0003, 81: 0.00

In [35]:

# Build the model (cluster the data)
gmm = GaussianMixture.train(parsedData, 2)

Py4JJavaError: An error occurred while calling o331.trainGaussianMixture.
: java.lang.OutOfMemoryError: Java heap space
	at scala.reflect.ManifestFactory$$anon$12.newArray(Manifest.scala:138)
	at scala.reflect.ManifestFactory$$anon$12.newArray(Manifest.scala:136)
	at breeze.linalg.DenseMatrix$mcD$sp.toDenseMatrix$mcD$sp(DenseMatrix.scala:231)
	at org.apache.spark.mllib.stat.distribution.MultivariateGaussian.calculateCovarianceConstants(MultivariateGaussian.scala:114)
	at org.apache.spark.mllib.stat.distribution.MultivariateGaussian.<init>(MultivariateGaussian.scala:61)
	at org.apache.spark.mllib.stat.distribution.MultivariateGaussian.<init>(MultivariateGaussian.scala:53)
	at org.apache.spark.mllib.clustering.GaussianMixture$$anonfun$3.apply(GaussianMixture.scala:154)
	at org.apache.spark.mllib.clustering.GaussianMixture$$anonfun$3.apply(GaussianMixture.scala:152)
	at scala.Array$.tabulate(Array.scala:331)
	at org.apache.spark.mllib.clustering.GaussianMixture.run(GaussianMixture.scala:1

In [36]:
points.take(2)

[LabeledPoint(1.0, (4125,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,89,90,91,92,93,94,95,96,97,99,100,101,102,103,104,105,106,107,108,109,110,111,112,114,115,116,117,118,122,218,275,295,447,490,571,572,600,622,725,761,815,870,923,958,1001,1015,1047,4123,4124],[123.0,1.31644,1.516003,5.605905,5.34676,0.013233,0.010729,0.091743,0.050768,3808.067871,702.992493,7533.133301,1390.499268,971.098511,1894.978027,114.965019,45.018257,0.635224,0.095226,0.063398,0.06121,0.038319,0.018285,0.011113,0.007736,0.004864,0.00422,0.003273,0.002699,0.002553,0.002323,0.002108,0.002036,0.001792,0.001553,0.00125,0.001317,0.001084,0.000818,0.000624,0.000586,0.000529,0.000426,0.000359,0.000446,0.000268,0.000221,0.000154,0.000217,0.000193,0.000163,0.000165,0.00021,0.000114,0.00013,5.5e-05,1.3e-05,0.733037,0.13

In [69]:
parsedData = points.groupWith(features)

In [70]:
parsedData.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 136.0 failed 1 times, most recent failure: Lost task 0.0 in stage 136.0 (TID 1030, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/bin/spark-1.4.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/bin/spark-1.4.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/bin/spark-1.4.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/bin/spark-1.4.0-bin-hadoop2.6/python/pyspark/rdd.py", line 1873, in <lambda>
    map_values_fn = lambda kv: (kv[0], f(kv[1]))
TypeError: 'Labele

In [63]:
weights = [0.98,0.02]
seed = 42
real,sample = parsedData.randomSplit(weights,seed)

In [65]:
sample.count()

1462034

In [66]:
gmm = GaussianMixture.train(sample, 2)

Py4JJavaError: An error occurred while calling o491.trainGaussianMixture.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 135.0 failed 1 times, most recent failure: Lost task 0.0 in stage 135.0 (TID 1029, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/bin/spark-1.4.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/local/bin/spark-1.4.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/bin/spark-1.4.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/bin/spark-1.4.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/mllib/linalg.py", line 74, in _convert_to_vector
    raise TypeError("Cannot convert type %s into Vector" % type(